In [2]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
from typing import Literal, Optional

import pandas as pd
from entities import EM_CDS_TRACKER_DICT, FX_TRACKER_DICT
from utils import Backtest, load_trackers, clean_stale_prices
from pathlib import Path
from aamalha.utils.performance import Performance

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


# Trackers Statistics

In [2]:
fx = load_trackers(FX_TRACKER_DICT).rename(columns=lambda col: col + "_fx")
cds = load_trackers(EM_CDS_TRACKER_DICT).rename(columns=lambda col: col + "_cds")

trackers = pd.concat(
    [fx, cds],
    axis=1,
).fillna(method="ffill")
trackers = pd.concat(
    [clean_stale_prices(trackers[col].copy()) for col in trackers], axis=1
)
trackers = trackers[:"2024-09-30"]
trackers

,BRL_fx,CLP_fx,CNY_fx,COP_fx,CZK_fx,HUF_fx,IDR_fx,INR_fx,MXN_fx,MYR_fx,...,MYR_cds,MXN_cds,PAB_cds,PEN_cds,PHP_cds,QAR_cds,RUB_cds,SAR_cds,ZAR_cds,TRY_cds
date,,,,,,,,,,,,,,,,,,,,,
1999-01-04,100.000,100.000,NaN,NaN,100.000,100.000,NaN,NaN,100.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,100.007,100.002,NaN,NaN,99.997,99.958,NaN,NaN,100.041,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-06,100.011,100.026,NaN,NaN,99.973,99.922,NaN,NaN,100.073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-07,99.939,99.976,NaN,NaN,99.866,99.885,NaN,NaN,99.952,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-08,99.950,100.025,NaN,NaN,99.614,99.756,NaN,NaN,100.095,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24,294.680,95.672,110.072,143.596,135.784,164.753,264.538,148.946,223.964,202.817,...,114.67,120.78,112.56,114.69,112.88,110.36,NaN,112.94,130.77,132.52
2024-09-25,293.339,95.580,110.317,141.541,135.852,164.776,266.102,149.103,222.043,205.130,...,114.68,120.56,112.69,114.58,112.92,110.38,NaN,112.97,130.77,132.52
2024-09-26,295.941,96.958,110.369,141.557,135.818,164.245,264.629,148.866,221.324,204.923,...,114.69,120.72,113.05,114.70,112.93,110.47,NaN,113.04,131.02,132.72


In [15]:
(trackers / trackers.apply(lambda col: col[col.first_valid_index()]) - 1).filter(
    like="_cds", axis=1
).sort_index(axis=1).rename(
    columns=lambda col: col[:3]
).to_clipboard(excel=True)

In [3]:
perf_trackers = Performance(trackers)

In [17]:
df_perf = perf_trackers.table.T.copy()
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
df_perf.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

In [24]:
perf_trackers.rolling_sharpe.ARS_cds

date
1999-01-04         NaN
1999-01-05         NaN
1999-01-06         NaN
1999-01-07         NaN
1999-01-08         NaN
                ...   
2024-09-24    4.086709
2024-09-25    4.159463
2024-09-26    4.271292
2024-09-27    4.259607
2024-09-30    4.333285
Name: ARS_cds, Length: 6422, dtype: float64

In [5]:
DICT_COLS = {
    "count": "N obs",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}
trackers_rolling_sharpe = perf_trackers.rolling_sharpe.describe().T
trackers_rolling_sharpe["Início"] = perf_trackers.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
trackers_rolling_sharpe.rename(
    columns=DICT_COLS, index=lambda idx: f"{idx[:3]} ({idx[4:].upper()})"
).to_clipboard()

# Valor

In [7]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []
for file_path in FOLDER.glob("*VALUE-*.xlsx"):
    file_path.is_file()
    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]
    s_backtest.name = file_path.stem
    list_series.append(s_backtest)

df_backtest = pd.concat(
    [clean_stale_prices(col.copy()) for col in list_series], axis=1
).sort_index(ascending=True)
df_backtest = df_backtest.dropna(how="all")
df_backtest.iloc[0].loc[df_backtest.iloc[1].isna()] = np.nan
df_backtest

,VALUE-CDS-1,VALUE-CDS-12,VALUE-CDS-3,VALUE-CDS-6,VALUE-FX-1,VALUE-FX-12,VALUE-FX-3,VALUE-FX-6
date,,,,,,,,
2008-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-09-24,40.101330,81.780428,60.591496,92.368321,106.006844,201.120002,103.845973,131.203978
2024-09-25,39.593416,81.020411,59.661797,91.540382,107.476198,203.157501,105.216536,132.695103
2024-09-26,39.879510,81.411450,60.102466,91.918861,107.358805,201.838780,104.992474,132.663733


In [19]:
from bwbbgdl import GoGet
import statsmodels.api as sm


tickers = ["SPXT Index", "FXCTEM8 Index"]
g = GoGet(enforce_strict_matching=True)
benchmarks: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
)
benchmarks = benchmarks.pivot_table(index="date", columns="id")
benchmarks.columns = benchmarks.columns.droplevel(0)

N_DAYS = 1
exog = benchmarks.fillna(method="ffill").copy()
endog = np.log(df_backtest["VALUE-FX-PAIRED-12"]).diff(N_DAYS).dropna()
exog = np.log(exog).diff(N_DAYS).reindex(endog.dropna().index).fillna(method="ffill")
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog).fit()
print(model.summary())

In [ ]:
from bwbbgdl import GoGet
import statsmodels.api as sm


tickers = ["SPXT Index", "FXCTEM8 Index", "MXWD Index", "HYG Equity", "EREM5LD5 Index"]
g = GoGet(enforce_strict_matching=True)
benchmarks: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
)
benchmarks = benchmarks.pivot_table(index="date", columns="id")
benchmarks.columns = benchmarks.columns.droplevel(0)

N_DAYS = 1
exog = benchmarks.fillna(method="ffill").copy()
endog = np.log(df_backtest["VALUE-FX-PAIRED-12"]).diff(N_DAYS).dropna()
exog = np.log(exog).diff(N_DAYS).reindex(endog.dropna().index).fillna(method="ffill")
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog).fit()
print(model.summary())

In [20]:
df_backtest.to_clipboard(excel=True)


In [21]:
df_backtest

,VALUE-CDS-1,VALUE-CDS-12,VALUE-CDS-3,VALUE-CDS-6,VALUE-FX-1,VALUE-FX-12,VALUE-FX-3,VALUE-FX-6
date,,,,,,,,
2009-02-03,100.000000,NaN,100.000000,NaN,100.000000,NaN,100.000000,NaN
2009-02-04,99.932300,NaN,99.870016,NaN,101.409886,NaN,99.987692,NaN
2009-02-05,99.960182,NaN,99.979774,NaN,101.389313,NaN,99.018387,NaN
2009-02-06,100.066599,NaN,100.069889,NaN,101.734625,NaN,98.469117,NaN
2009-02-09,100.153646,NaN,100.006111,NaN,101.909729,NaN,98.939112,NaN
...,...,...,...,...,...,...,...,...
2024-09-24,40.101330,81.780428,60.591496,92.368321,106.006844,201.120002,103.845973,131.203978
2024-09-25,39.593416,81.020411,59.661797,91.540382,107.476198,203.157501,105.216536,132.695103
2024-09-26,39.879510,81.411450,60.102466,91.918861,107.358805,201.838780,104.992474,132.663733


## Backtests

In [22]:
performance_1y = Performance(df_backtest)
performance_5y = Performance(df_backtest, rolling_window=252*5)

## Estatísticas Descritivas dos  Trackers - Retornos Diários									

In [23]:
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
performance_1y.table.T.rename(columns=DICT_COLS).to_clipboard()

## Sharpe Média Móvel 1 Ano														

In [24]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_1y_rolling_sharpe = performance_1y.rolling_sharpe.describe().T
performance_1y_rolling_sharpe["Início"] = performance_1y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_1y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [26]:
rolling_sharpe = performance_1y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N + 1)]
col = performance_1y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

## Sharpe Média Móvel 5 Ano

In [25]:
DICT_COLS = {
    "count": "N",
    "mean": "Média",
    "std": "DP",
    "min": "Min",
    "max": "Max",
}

performance_5y_rolling_sharpe = performance_5y.rolling_sharpe.describe().T
performance_5y_rolling_sharpe["Início"] = performance_5y.rolling_sharpe.apply(
    lambda col: col.first_valid_index()
)
performance_5y_rolling_sharpe.rename(columns=DICT_COLS).to_clipboard()

In [27]:
# rolling_sharpe_quantiles

rolling_sharpe = performance_5y.rolling_sharpe.copy()

N = 500
percentils = [n / N for n in range(0, N +1)]
col = performance_5y.rolling_sharpe.iloc[:, 0]
pd.concat(
    [
        pd.Series(
            {p: rolling_sharpe[col].quantile(p) for p in percentils},
            name=col,
        )
        for col in rolling_sharpe.columns
    ],
    axis=1,
).to_clipboard()

# old

In [ ]:
data = df_backtest.copy()
def export_latex_annual_returns(data: pd.DataFrame): 
    INDEX_NAME = "Ano"
    COLUMN_NAME = "Backtests"
    CAPTION = "Retornos Anuais dos Backtests"
    SOURCE = "Pedro Monzú, 2024"
    LABEL = "annual_returns"

    output = (
        "\\begin{table}[htbp]\n"
        "\t\centering\n"
        "\t\caption{%s}\n"
        "\n\t%s\n"
        "\t\\vspace{0.3cm}\n"
        "\t\label{tab:%s}\n"
        "\t\source{%s}\n"
        "\end{table}\n"
    )

    df_year_returns = (
        np.exp(np.log(data).diff(1).groupby(data.index.to_period("Y")).sum()) - 1
    )
    df_year_returns.index.name = INDEX_NAME
    df_year_returns.columns.name = COLUMN_NAME

    formatted_df = df_year_returns.applymap(
        lambda x: f"({abs(x):.2%})" if x < 0 else f"{x:.2%}"
    )
    formatted_df = formatted_df.replace(np.nan, "-")
    latex_output = formatted_df.to_latex(
        escape=False, column_format="c" * (len(formatted_df.columns) + 1)
    ).replace("%", "\%").replace("\n", "\n\t")
    return output % (CAPTION, latex_output, LABEL, SOURCE)

print(export_latex_annual_returns(data))

In [61]:
performance = Performance(data)


In [ ]:
x = -1.2345
precision = 7
percentage = True
parentheses = False
formatted_string = f"{'('if parentheses else '-'}{abs(x):.{precision}{'%' if percentage else 'f'}}{')'if parentheses else ''}"
formatted_string

In [ ]:
x = np.nan
x is np.nan

In [293]:
def float_to_string(value: float, pattern: str) -> str:
    # Parentheses
    parentheses == pattern[0] == "(" and pattern[-1] == ")"

    # Decimals
    match = re.search(r"\.(\d+)", pattern)
    decimals = len(match.group(1)) if match else 0

    # Percentage
    percentage = format_pos[-1] == "%"

    # Thousands Divisor
    pattern_aux = pattern[: -1 if percentage else None]
    match = re.search(r"(\,+)$", pattern_aux)
    divisor_thousands = len(match.group(1)) if match else 0

    # Integer part
    pattern_aux = pattern.split(".")[0].rstrip("%,")
    pos_last_zero = pattern_aux.rfind("0")
    integer_part = max(1, pattern_aux.count("0"))

    # Thousands Separator
    comma_separator = "," in pattern_aux  # TODO: improve the separators part

    # Adjusted Value
    adjusted_value = value / (1000 ** (divisor_thousands))

    # String Length
    decimal_length = decimals + 1 if decimals else 0
    percentage_length = 1 if percentage else 0
    value_aux = adjusted_value * (100 if percentage else 1)
    commas_length = int(np.log10(value_aux) // 3) if comma_separator else 0
    string_length = integer_part + decimal_length + percentage_length + commas_length

    formatted_value = f"{adjusted_value:0{string_length}{',' if comma_separator else ''}.{decimals}{'%' if percentage else 'f'}}"
    formatted_value = f"({formatted_value})" if parentheses else formatted_value
    return formatted_value

In [ ]:
import re

x = 123456.78911111111
format_template = ",000,000,000.000%;(#,##0.00,%);-"


def formatter(value: float, format_template) -> str:
    formats = format_template.split(";")
    format_pos = formats[0]
    format_neg = format_pos if len(formats) < 2 else formats[1]
    format_missing = format_pos if len(formats) == 3 else ""

    if value is np.nan:
        return format_missing
    else:
        return float_to_string(value, format_pos if value >= 0 else format_neg)

formatter(x, format_template)

In [ ]:
from typing import Optional, Union


def format_float(
    data: Union[pd.Series, pd.DataFrame],
    precision: Optional[int] = 0,
    percentage: bool = False,
    parentheses: bool = True,
    missing: [str] = "-",
):
    formatting = lambda x: (
        f"{'('if parentheses else '-'}"  # only if x <0 else ''
        f"{abs(x):.{precision}{'%' if percentage else 'f'}}"
        f"{')'if parentheses else ''}"  # only if x <0 else ''
    )
    return data.applymap(formatting).replace(np.nan, missing)


df_perf = performance.table.T.copy()
DICT_COLS = {
    "Return": "Retorno",
    "Vol": "Volatilidade",
    "Skew": "Assimetria",
    "Kurt": "Curtose",
    "Sharpe": "Sharpe",
    "Sortino": "Sortino",
    "DD 1%q": "DD (1%)",
    "Max DD": "Max DD",
    "Start Date": "Início",
}
COLS_FLOAT_PCT = [
    "Retorno",
    "Volatilidade",
    "Drawdown (1º Percentil)",
    "Máximo Drawdown",
]
COLS_FLOAT_DEFAULT = [
    "Índice de Sharpe",
    "Assimetria",
    "Curtose",
    "Índice de Sortino",
]
DATE_COLS = [
    "Início da Série",
]
df_perf[COLS_FLOAT_PCT] = format_float(
    df_perf[COLS_FLOAT_PCT], precision=2, percentage=True, parentheses=True, missing="-"
)
df_perf[COLS_FLOAT_DEFAULT] = format_float(
    df_perf[COLS_FLOAT_DEFAULT],
    precision=3,
    percentage=False,
    parentheses=True,
    missing="-",
)
df_perf = df_perf.rename(columns=DICT_COLS)
df_perf